In [1]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to C:\Users\Neel
[nltk_data]     Rami\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Neel
[nltk_data]     Rami\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords

In [3]:
from collections import defaultdict

In [4]:
from string import punctuation

In [5]:
from heapq import nlargest

In [6]:
class FrequencySummarizer:
    
    def __init__(self, min_cut=0.1, max_cut=0.9):
        self._min_cut = min_cut
        self._max_cut = max_cut 
        self._stopwords = set(stopwords.words('english') + list(punctuation))
    
    def _compute_frequencies(self, word_sent):
        freq = defaultdict(int)
        for s in word_sent:
            for word in s:
                if word not in self._stopwords:
                    freq[word] += 1
        m = float(max(freq.values()))
        for w in list(freq.keys()):
            freq[w] = freq[w]/m
            if freq[w] >= self._max_cut or freq[w] <= self._min_cut:
                del freq[w]
        return freq
    
    def summarize(self, text, n):
        sents = sent_tokenize(text)
        assert n <= len(sents)
        word_sent = [word_tokenize(s.lower()) for s in sents]
        self._freq = self._compute_frequencies(word_sent)
        ranking = defaultdict(int)
        for i,sent in enumerate(word_sent):
            for w in sent:
                if w in self._freq:
                    ranking[i] += self._freq[w]
        sents_idx = nlargest(n, ranking, key=ranking.get)
        return [sents[j] for j in sents_idx]

In [7]:
import urllib.request
from bs4 import BeautifulSoup

In [64]:
def get_text_washington_post(url):
    page = urllib.request.urlopen(url).read().decode('utf8')
    soup = BeautifulSoup(page,"html.parser")
    text = ' '.join(map(lambda p: p.text, soup.find_all('article')))
    return soup.title.text, text

In [65]:
url="https://www.washingtonpost.com/news/the-switch/wp/2015/08/06/why-kids-are-meeting-more-strangers-online-than-ever-before/"

In [66]:
textOfurl1=get_text_washington_post(url)

In [62]:
print(textOfurl1)

('Third of all teens who meet strangers online are meeting them in person, too - The Washington Post', '')


In [67]:
fs = FrequencySummarizer()
summary = fs.summarize(textOfurl1[1], 3)

In [68]:
print(textOfurl1[0])
print(summary)

Third of all teens who meet strangers online are meeting them in person, too - The Washington Post
['"The digital world has taken its place alongside school and friends\' houses and extracurriculars as a place where teens go to make and strengthen friendships," said Amanda Lenhart, author of the report "Teens, Technology & Friendships" and an associate\xa0director of research at Pew.', '"Young people are very aware that people have highly curated images and that text fights can quickly go out of control and they are trying to sort it all out," said Rosalind Wiseman, author of "Queen Bees and Wannabes" and speaker on youth issues.', "Nearly half of those surveyed say they've at least occasionally seen posts about events that they were invited to; and\xa085 percent said they think social media users present a carefully crafted image of themselves online that may not be authentic."]
